### Import dependencies and read in csv files

In [1]:
# import dependencies
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt    

In [2]:
# read in csv
mouse_file = "../../Downloads/WorkingDirectory/gt-virt-data-pt-12-2021-u-c/05-Matplotlib/Homework/Instructions/Pymaceuticals/data/Mouse_metadata.csv"
study_file = "../../Downloads/WorkingDirectory/gt-virt-data-pt-12-2021-u-c/05-Matplotlib/Homework/Instructions/Pymaceuticals/data/Study_results.csv"

mouse_master = pd.read_csv(mouse_file)
study_master = pd.read_csv(study_file)


### Interrogate data and clean

In [3]:
# view mouse columns
mouse_master.columns

Index(['Mouse ID', 'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)'], dtype='object')

In [4]:
# view mouse data
mouse_master.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [5]:
# view study columns
study_master.columns

Index(['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites'], dtype='object')

In [6]:
# view study data
study_master.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [7]:
mouse_master.count()

Mouse ID        249
Drug Regimen    249
Sex             249
Age_months      249
Weight (g)      249
dtype: int64

In [8]:
# count number unique mice in the mouse file
mouse_master["Mouse ID"].nunique()

249

In [9]:
study_master.count()

Mouse ID              1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [10]:
# count number unique mice in the study file
study_master["Mouse ID"].nunique()

249

In [11]:
# count null values in each column

# null["null count"] = study_master["Timepoint"].isna().sum()
# null

# def null_cnt(df_input, column_input, column_output):
#     df_input[column_output] = df_input[column_input].isna().sum()
#     return df_input[column_output]

# null_cnt(study_master, "Timepoint", "null")

In [25]:
#calculate the number of null values for each column
print("The number of null values by feature:")

def print_null_msg(df_input, textdf, textfeature, column_output):
    return print(f"{textdf} -- {textfeature}: {df_input[textfeature].isna().sum().sum()}")

print_null_msg(study_master, "study_master", "Timepoint", "null count")
print_null_msg(study_master, "study_master", "Tumor Volume (mm3)", "null count")
print_null_msg(study_master, "study_master", "Metastatic Sites", "null count")
print_null_msg(mouse_master, "mouse_master", "Drug Regimen", "null count")
print_null_msg(mouse_master, "mouse_master", "Sex", "null count")
print_null_msg(mouse_master, "mouse_master", "Age_months", "null count")
print_null_msg(mouse_master, "mouse_master", "Weight (g)", "null count")

The number of null values by feature:
study_master -- Timepoint: 0
study_master -- Tumor Volume (mm3): 0
study_master -- Metastatic Sites: 0
mouse_master -- Drug Regimen: 0
mouse_master -- Sex: 0
mouse_master -- Age_months: 0
mouse_master -- Weight (g): 0


In [13]:
# left merge study to mouse_master, mouse_master, how = "left", on = "Mouse ID")
mouse_combined = pd.merge(mouse_master, study_master, how = "outer", on = "Mouse ID")
mouse_combined.count()

Mouse ID              1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [14]:
# calculate the number of mice
unique_mice = mouse_combined["Mouse ID"].nunique()
unique_cnt_df = pd.DataFrame([unique_mice]).rename(columns = {0: "Unique Mice"})
unique_cnt_df

,Unique Mice
0,249


In [15]:
# id duplicate Mouse ID at Timepoint level
#mouse_combined["tm_str"] = mouse_combined["Timepoint"].astype(str)
mouse_combined["idt_key"] = mouse_combined["Mouse ID"] + (mouse_combined["Timepoint"].astype(str))
keys = mouse_combined["idt_key"].value_counts()
keys_df = pd.DataFrame(keys)
keys_df = keys_df.rename(columns = {"idt_key": "cnt"})
dupkeys_df = keys_df.loc[keys_df["cnt"] > 1, :]
dupkeys_df.reset_index(inplace = True)
dupkeys_df = dupkeys_df.rename(columns = {"index": "idt_key"})
dupkeys_df

,idt_key,cnt
0,g98920,2
1,g98915,2
2,g98910,2
3,g9895,2
4,g9890,2


In [16]:
# get all of the data for the duplicate mice, creating a dataframe
duplicate_idt_keys = pd.merge(dupkeys_df, mouse_combined, how = "left", on = "idt_key")
duplicate_mouseid = duplicate_idt_keys["Mouse ID"]
duplicate_mouse = pd.merge(duplicate_mouseid, mouse_combined, how = "left", on = "Mouse ID")
dup_mouseid_val = duplicate_mouse["Mouse ID"][0]
duplicate_mouse_data = mouse_combined.loc[mouse_combined["Mouse ID"] == dup_mouseid_val, :]
duplicate_mouse_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,idt_key
908,g989,Propriva,Female,21,26,0,45.000000,0,g9890
909,g989,Propriva,Female,21,26,0,45.000000,0,g9890
910,g989,Propriva,Female,21,26,5,48.786801,0,g9895
911,g989,Propriva,Female,21,26,5,47.570392,0,g9895
912,g989,Propriva,Female,21,26,10,51.745156,0,g98910
913,g989,Propriva,Female,21,26,10,49.880528,0,g98910
914,g989,Propriva,Female,21,26,15,51.325852,1,g98915
915,g989,Propriva,Female,21,26,15,53.442020,0,g98915
916,g989,Propriva,Female,21,26,20,55.326122,1,g98920
917,g989,Propriva,Female,21,26,20,54.657650,1,g98920


In [17]:
# create a dataframe without the duplicates
nondups_df = mouse_combined.loc[mouse_combined["Mouse ID"] != dup_mouseid_val, :]
nondups_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,idt_key
0,k403,Ramicane,Male,21,16,0,45.000000,0,k4030
1,k403,Ramicane,Male,21,16,5,38.825898,0,k4035
2,k403,Ramicane,Male,21,16,10,35.014271,1,k40310
3,k403,Ramicane,Male,21,16,15,34.223992,1,k40315
4,k403,Ramicane,Male,21,16,20,32.997729,1,k40320
...,...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2,z96925
1889,z969,Naftisol,Male,9,30,30,65.841013,3,z96930
1890,z969,Naftisol,Male,9,30,35,69.176246,4,z96935
1891,z969,Naftisol,Male,9,30,40,70.314904,4,z96940


In [18]:
# count unique mice in clean df
unique_mice_cnt = nondups_df["Mouse ID"].nunique()
unique_mice_cnt_df = pd.DataFrame([unique_mice_cnt]).rename(columns = {0: "Unique Mouse Count"})
unique_mice_cnt_df

,Unique Mouse Count
0,248


### Calculate summary statistics 

In [19]:
# use agg function to calculate the mean, median var, std dev, sem
# use .columns to force rename of columns and the reset the index to flatten from multi index after groupby creates multi index
# calculate std error of the mean: std dev/sq rt(N)
nondups_grp_summary_df = nondups_df.groupby(["Drug Regimen"]).agg({"Tumor Volume (mm3)": ["count", "mean", "median", "var", "std", "sem"]})
nondups_grp_summary_df.columns = ["Count", "Tumor Avg", "Tumor Median", "Tumor Variance", "Tumor Std Dev", "Tumor SEM"]
nondups_grp_summary_df.reset_index(inplace = True)
nondups_grp_summary_df.set_index("Drug Regimen", inplace = True)
nondups_grp_summary_df

,Count,Tumor Avg,Tumor Median,Tumor Variance,Tumor Std Dev,Tumor SEM
Drug Regimen,,,,,,
Capomulin,230,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,178,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,178,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,188,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,186,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,181,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,148,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,228,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,181,54.233149,52.431737,59.450562,7.710419,0.573111


In [23]:
# calculate summary overall using agg function, similar to approach calculating at drug regimen level
# clean up df, renaming column header and index label
nondups_totsummary_df = nondups_df.agg({"Tumor Volume (mm3)": ["count", "mean", "median", "var", "std", "sem"]}).transpose()
nondups_totsummary_df.columns = ["Count", "Tumor Avg", "Tumor Median", "Tumor Variance", "Tumor Std Dev", "Tumor SEM"]
nondups_totsummary_df.rename(index = {'Tumor Volume (mm3)': "Drug Regimen Summary"}, inplace = True)
nondups_totsummary_df

,Count,Tumor Avg,Tumor Median,Tumor Variance,Tumor Std Dev,Tumor SEM
Drug Regimen Summary,1880.0,50.435293,48.933454,79.462907,8.914197,0.205591


In [24]:
# combine both drug regimen and overall summary dfs using append function
nondups_totsummary_combined_df = nondups_grp_summary_df.append(nondups_totsummary_df)
nondups_totsummary_combined_df

,Count,Tumor Avg,Tumor Median,Tumor Variance,Tumor Std Dev,Tumor SEM
Capomulin,230.0,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,178.0,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,178.0,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,188.0,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,186.0,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,181.0,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,148.0,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,228.0,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,181.0,54.233149,52.431737,59.450562,7.710419,0.573111
Zoniferol,182.0,53.236507,51.818479,48.533355,6.966589,0.516398
